In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


In [4]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00


In [5]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 59.2 MB/s eta 0:00:00


# Requirements

In [6]:
import torch

from torch.optim import Adam
from torch_geometric.nn import ComplEx, DistMult, RotatE, TransE
from torch_geometric.data import Data

from base import OptimizerConfig, cross_validation
from base import SimplePytorchData, SimplePytorchDataTrainTestSplit
from base import SimpleTrainer, SimpleTester
from src.config import SimpleClassifierConfig, GraphAutoEncoderConfig, KGEConfig
from src.features import get_relations, get_entities, get_associations, get_homogeneous_graph, get_kge_pair_embedd_for_training_data
from src.models import SimpleMDAClassifier, SimpleMDAClassifierFactory
from src.utils import train_test_sampler, prj_logger
from torch_geometric.nn import GCNConv

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

# TransE

## Config

In [9]:
kge_optimizer_config = OptimizerConfig()
kge_optimizer_config.optimizer = torch.optim.Adam
kge_optimizer_config.lr = 0.01
kge_optimizer_config.batch_size = 1000
kge_optimizer_config.n_epoch = 100
kge_optimizer_config.exp_name = "Optimizer for Graph Auto Encoder"
kge_optimizer_config.device = device
kge_optimizer_config.report_size = device

In [10]:
kge_model_config = KGEConfig()
kge_model_config.kge = TransE
kge_model_config.hidden_channels = 32

## Embedding

In [11]:
md_embed = get_kge_pair_embedd_for_training_data(kge_model_config, kge_optimizer_config)

2024-02-11 13:29:29,359 [INFO] Calling get_node2vec_pair_embedd on cpu device ...
2024-02-11 13:29:29,361 [INFO] Calling get_homogeneous_graph
2024-02-11 13:29:31,087 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])
2024-02-11 13:29:31,097 [INFO] Calling get_kge_embedd on cpu device ...
2024-02-11 13:29:31,099 [INFO] Calling get_knowledge_graph
2024-02-11 13:29:32,360 [INFO] knowledge graph data : Data(num_nodes=66911, edge_index=[2, 633662], edge_type=[633662])
2024-02-11 13:29:32,392 [INFO] Setting num relations and num nodes for kge config to 39 and 66911
2024-02-11 13:29:32,395 [INFO] Creating KGE model ...
2024-02-11 13:29:32,398 [INFO] Initialing MDATransE with model_config {'model_name': None}
2024-02-11 13:29:32,471 [INFO] Training KGE ...
2024-02-11 13:29:32,473 [INFO] Running KGETrainer with Optimizer for Graph Auto Encoder
2024-02-11 13:29:32,475 [INFO] Creating <class 'torch.optim.adam.Adam'> with lr : 0.01
2024-02-11 13:29:32,477 [INFO] moving model to 

# Classification

## Data

In [12]:
associations = get_associations()
y = torch.tensor(associations['increased'].tolist(), dtype=torch.float32).reshape(-1, 1).to(device)

In [13]:
# Train Test Split
train_indices, test_indices = train_test_sampler(y.shape[0], 0.7)

data = SimplePytorchData(md_embed, y)
train_data = SimplePytorchData(md_embed[train_indices], y[train_indices])
test_data = SimplePytorchData(md_embed[test_indices], y[test_indices])

2024-02-11 14:09:33,242 [INFO] Initializing SimplePytorchData with X shape : torch.Size([898, 64]) and y shape : torch.Size([898, 1])
2024-02-11 14:09:33,246 [INFO] Initializing SimplePytorchData with X shape : torch.Size([628, 64]) and y shape : torch.Size([628, 1])
2024-02-11 14:09:33,248 [INFO] Initializing SimplePytorchData with X shape : torch.Size([270, 64]) and y shape : torch.Size([270, 1])


## Classifier

In [14]:
simple_classifier_config = SimpleClassifierConfig()
simple_classifier_config.model_name = "simple classifier"
simple_classifier_config.input_dim = md_embed.shape[1]
simple_classifier_config.hidden_dim = 32
simple_classifier_config.output_dim = 1
simple_classifier_config.num_layers = 2
simple_classifier_config.dropout = 0.1

In [15]:
mda_classifier = SimpleMDAClassifier(simple_classifier_config)

2024-02-11 14:09:33,271 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-11 14:09:33,274 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout


## Optimizer

In [16]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.optimizer = torch.optim.Adam
classifier_optimizer_config.criterion = torch.nn.BCEWithLogitsLoss()
classifier_optimizer_config.lr = 0.01
classifier_optimizer_config.batch_size = 32
classifier_optimizer_config.n_epoch = 50
classifier_optimizer_config.exp_name = "adam optimizer"
classifier_optimizer_config.save = False
classifier_optimizer_config.save_path = None
classifier_optimizer_config.device = device
classifier_optimizer_config.report_size = 10  # batch to report ratio
classifier_optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [17]:
train_result = SimpleTrainer().train(model=mda_classifier,
                                     data=train_data,
                                     config=classifier_optimizer_config)

2024-02-11 14:09:33,296 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-11 14:09:33,299 [INFO] moving data and model to cpu
2024-02-11 14:09:33,362 [INFO] loss: 0.0219    [1,    10]
2024-02-11 14:09:33,382 [INFO] loss: 0.0177    [1,    20]
2024-02-11 14:09:33,403 [INFO] loss: 0.0176    [2,    10]
2024-02-11 14:09:33,421 [INFO] loss: 0.0157    [2,    20]
2024-02-11 14:09:33,446 [INFO] loss: 0.0161    [3,    10]
2024-02-11 14:09:33,470 [INFO] loss: 0.0140    [3,    20]
2024-02-11 14:09:33,493 [INFO] loss: 0.0139    [4,    10]
2024-02-11 14:09:33,517 [INFO] loss: 0.0140    [4,    20]
2024-02-11 14:09:33,547 [INFO] loss: 0.0120    [5,    10]
2024-02-11 14:09:33,566 [INFO] loss: 0.0139    [5,    20]
2024-02-11 14:09:33,588 [INFO] loss: 0.0133    [6,    10]
2024-02-11 14:09:33,609 [INFO] loss: 0.0132    [6,    20]
2024-02-11 14:09:33,631 [INFO] loss: 0.0123    [7,    10]
2024-02-11 14:09:33,651 [INFO] loss: 0.0124    [7,    20]
2024-02-11 14:09:33,671 [INFO] loss: 0.0107  

### Test

In [18]:
test_result = SimpleTester().test(model=mda_classifier,
                                  data=test_data,
                                  config=classifier_optimizer_config)

2024-02-11 14:09:35,437 [INFO] Running Simple Tester with config : adam optimizer
2024-02-11 14:09:35,440 [INFO] moving data and model to cpu
2024-02-11 14:09:35,468 [INFO] Result on Test Data : {'AUC': 0.9743251415690802, 'ACC': 0.8962962962962963, 'F1 Score': 0.8962450592885376, 'AUPR': 0, 'Loss': 0.24296143651008606}


In [19]:
test_result.get_result()

{'AUC': 0.9743251415690802,
 'ACC': 0.8962962962962963,
 'F1 Score': 0.8962450592885376,
 'AUPR': 0,
 'Loss': 0.24296143651008606}

## Cross Validation

In [20]:
trainer = SimpleTrainer()
tester = SimpleTester()
factory = SimpleMDAClassifierFactory(simple_classifier_config)
spliter = SimplePytorchDataTrainTestSplit(data)
cross_validation(k=5, data_size=data.X.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-11 14:09:35,492 [INFO] Initializing SimpleMDAClassifierFactory with model : simple classifier
2024-02-11 14:09:35,493 [INFO] Initializing SimplePytorchDataTrainTestSplit
2024-02-11 14:09:35,495 [INFO] Start 5-fold Cross Validation with config : adam optimizer
2024-02-11 14:09:35,497 [INFO] ---- Fold 1 ----
2024-02-11 14:09:35,500 [INFO] Initializing SimplePytorchData with X shape : torch.Size([719, 64]) and y shape : torch.Size([719, 1])
2024-02-11 14:09:35,501 [INFO] Initializing SimplePytorchData with X shape : torch.Size([179, 64]) and y shape : torch.Size([179, 1])
2024-02-11 14:09:35,502 [INFO] Initializing SimpleMDAClassifier with model : simple classifier
2024-02-11 14:09:35,504 [INFO] Initial SimpleMLP with 64 input dimension, 32 hidden dimension, 1 
            output dimension, 2 layers and with 0.1 dropout
2024-02-11 14:09:35,506 [INFO] Running Simple Trainer with config : adam optimizer
2024-02-11 14:09:35,507 [INFO] moving data and model to cpu
2024-02-11 14:09:35,